we have an image dataset in directory clean_keypoints_dataset. for each image we have a labelme-format annotation json containing labeled points and sets of labeled points(line-strip). The images for 1920,1080. The task is to flip each image horizontally and not only that, to also create an annotation json for the flipped image. For example, for image 33.jpg and its afferent annotation file 33.json, we will create 33_h.jpg(horizontally flipped image) and also 33_h.json. Inside the json you will flip all anottated keypoints (x,y), as well as the ones for each line-strip(e.g. left-9m).
for mirroring the labels of points when you flip them, these are the correspondencies:
1-3
17-19
13-15
23-25
24-26
9-11
10-12
5-7
6-8
21-21
27-27
28-28
22-22
14-16
2-4
18-4
the labels will be strings but these are the ints.
for lines, you will have labels like (left-9m, right-6m). For these, when you flip the points, the new label will be to just replace left with right or right with left in the initial label name.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Let's start by exploring the dataset structure:

```python
from pathlib import Path
import json
from PIL import Image

data_dir = Path('clean_keypoints_dataset')
img_files = sorted(data_dir.glob('*.jpg'))
json_files = sorted(data_dir.glob('*.json'))
len(img_files), len(json_files)
```

In [ ]:
from pathlib import Path
import json
from PIL import Image

data_dir = Path('clean_keypoints_dataset')
img_files = sorted(data_dir.glob('*.jpg'))
json_files = sorted(data_dir.glob('*.json'))
len(img_files), len(json_files)

(115, 79)

Also, important to notice that some images do not have the annotation file(.json). You can skip flipping those

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Let's look at an example JSON file to understand the structure:

```python
sample_json = json_files[0]
sample_data = json.loads(sample_json.read_text())
sample_json.name, sample_data.keys()
```

In [ ]:
sample_json = json_files[0]
sample_data = json.loads(sample_json.read_text())
sample_json.name, sample_data.keys()

('1.json',
 dict_keys(['version', 'flags', 'shapes', 'imagePath', 'imageData', 'imageHeight', 'imageWidth']))

Inside the shapes value you will find basically all annotations(points and liens)

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Let's examine the shapes structure:

```python
sample_data['shapes'][:3]
```

In [ ]:
sample_data['shapes'][:3]

[{'label': '1',
  'points': [[626.5, 370.3]],
  'group_id': None,
  'description': '',
  'shape_type': 'point',
  'flags': {},
  'mask': None},
 {'label': '17',
  'points': [[852.4, 387.4]],
  'group_id': None,
  'description': '',
  'shape_type': 'point',
  'flags': {},
  'mask': None},
 {'label': '13',
  'points': [[554.5, 412.5]],
  'group_id': None,
  'description': '',
  'shape_type': 'point',
  'flags': {},
  'mask': None}]

also forgot to mention that there will be two lines(labels: "up", "down"). When flipping those you maintain the label

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Let's check for line-strip shapes in the data:

```python
[s for s in sample_data['shapes'] if s['shape_type'] == 'linestrip'][:2]
```

In [ ]:
[s for s in sample_data['shapes'] if s['shape_type'] == 'linestrip'][:2]

[{'label': 'up',
  'points': [[629.1, 370.4],
   [711.2280701754386, 376.14035087719293],
   [798.7134502923976, 382.9239766081871],
   [875.2, 388.3],
   [970.8, 395.5],
   [1082.5609756097563, 402.8048780487805],
   [1224.3902439024391, 413.2926829268293],
   [1345.4878048780488, 421.0975609756098],
   [1509.878048780488, 432.80487804878055],
   [1755.609756097561, 448.4146341463415],
   [1913.1707317073171, 457.56097560975616]],
  'group_id': None,
  'description': '',
  'shape_type': 'linestrip',
  'flags': {},
  'mask': None},
 {'label': 'left-7m',
  'points': [[899.7596153846154, 612.2355769230769],
   [892.6682692307692, 618.125],
   [886.0576923076923, 624.1346153846154],
   [880.0480769230769, 629.4230769230769]],
  'group_id': None,
  'description': '',
  'shape_type': 'linestrip',
  'flags': {},
  'mask': None}]

you got it

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Let's create the label mapping and helper functions:

```python
label_map = {1:3, 3:1, 17:19, 19:17, 13:15, 15:13, 23:25, 25:23, 24:26, 26:24, 9:11, 11:9, 10:12, 12:10, 5:7, 7:5, 6:8, 8:6, 21:21, 27:27, 28:28, 22:22, 14:16, 16:14, 2:4, 4:2, 18:20, 20:18}

def flip_x(x, w=1920): return w - x
def flip_label(lbl):
    if lbl in ['up', 'down']: return lbl
    if 'left' in lbl: return lbl.replace('left', 'right')
    if 'right' in lbl: return lbl.replace('right', 'left')
    return str(label_map.get(int(lbl), int(lbl)))

flip_label('left-7m'), flip_label('up'), flip_label('17')
```

In [ ]:
label_map = {1:3, 3:1, 17:19, 19:17, 13:15, 15:13, 23:25, 25:23, 24:26, 26:24, 9:11, 11:9, 10:12, 12:10, 5:7, 7:5, 6:8, 8:6, 21:21, 27:27, 28:28, 22:22, 14:16, 16:14, 2:4, 4:2, 18:20, 20:18}

def flip_x(x, w=1920): return w - x
def flip_label(lbl):
    if lbl in ['up', 'down']: return lbl
    if 'left' in lbl: return lbl.replace('left', 'right')
    if 'right' in lbl: return lbl.replace('right', 'left')
    return str(label_map.get(int(lbl), int(lbl)))

flip_label('left-7m'), flip_label('up'), flip_label('17')

('right-7m', 'up', '19')

everything looks good. You can start the flipping

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

```python
def flip_shape(shape, w=1920):
    flipped = shape.copy()
    flipped['label'] = flip_label(shape['label'])
    flipped['points'] = [[flip_x(x, w), y] for x,y in shape['points']]
    return flipped

def flip_annotation(json_path):
    data = json.loads(json_path.read_text())
    img_path = json_path.parent / data['imagePath']
    if not img_path.exists(): return None
    
    img = Image.open(img_path)
    flipped_img = img.transpose(Image.FLIP_LEFT_RIGHT)
    
    stem = json_path.stem
    flipped_img_path = json_path.parent / f"{stem}_h.jpg"
    flipped_json_path = json_path.parent / f"{stem}_h.json"
    
    flipped_img.save(flipped_img_path)
    
    flipped_data = data.copy()
    flipped_data['imagePath'] = flipped_img_path.name
    flipped_data['shapes'] = [flip_shape(s, data['imageWidth']) for s in data['shapes']]
    flipped_data['imageData'] = None
    
    flipped_json_path.write_text(json.dumps(flipped_data, indent=2))
    return flipped_img_path, flipped_json_path

for jf in json_files:
    result = flip_annotation(jf)
    if result: print(f"Created {result[0].name} and {result[1].name}")
```

In [ ]:
def flip_shape(shape, w=1920):
    flipped = shape.copy()
    flipped['label'] = flip_label(shape['label'])
    flipped['points'] = [[flip_x(x, w), y] for x,y in shape['points']]
    return flipped

def flip_annotation(json_path):
    data = json.loads(json_path.read_text())
    img_path = json_path.parent / data['imagePath']
    if not img_path.exists(): return None
    
    img = Image.open(img_path)
    flipped_img = img.transpose(Image.FLIP_LEFT_RIGHT)
    
    stem = json_path.stem
    flipped_img_path = json_path.parent / f"{stem}_h.jpg"
    flipped_json_path = json_path.parent / f"{stem}_h.json"
    
    flipped_img.save(flipped_img_path)
    
    flipped_data = data.copy()
    flipped_data['imagePath'] = flipped_img_path.name
    flipped_data['shapes'] = [flip_shape(s, data['imageWidth']) for s in data['shapes']]
    flipped_data['imageData'] = None
    
    flipped_json_path.write_text(json.dumps(flipped_data, indent=2))
    return flipped_img_path, flipped_json_path

for jf in json_files:
    result = flip_annotation(jf)
    if result: print(f"Created {result[0].name} and {result[1].name}")

Created 1_h.jpg and 1_h.json
Created 10_h.jpg and 10_h.json
Created 100_h.jpg and 100_h.json
Created 101_h.jpg and 101_h.json
Created 102_h.jpg and 102_h.json
Created 103_h.jpg and 103_h.json
Created 104_h.jpg and 104_h.json
Created 105_h.jpg and 105_h.json
Created 106_h.jpg and 106_h.json


Created 107_h.jpg and 107_h.json
Created 108_h.jpg and 108_h.json
Created 109_h.jpg and 109_h.json
Created 11_h.jpg and 11_h.json
Created 110_h.jpg and 110_h.json
Created 111_h.jpg and 111_h.json
Created 112_h.jpg and 112_h.json
Created 113_h.jpg and 113_h.json
Created 114_h.jpg and 114_h.json
Created 12_h.jpg and 12_h.json


Created 13_h.jpg and 13_h.json
Created 14_h.jpg and 14_h.json
Created 15_h.jpg and 15_h.json
Created 16_h.jpg and 16_h.json
Created 17_h.jpg and 17_h.json
Created 18_h.jpg and 18_h.json
Created 19_h.jpg and 19_h.json
Created 2_h.jpg and 2_h.json
Created 20_h.jpg and 20_h.json


Created 21_h.jpg and 21_h.json
Created 22_h.jpg and 22_h.json
Created 23_h.jpg and 23_h.json
Created 24_h.jpg and 24_h.json
Created 25_h.jpg and 25_h.json
Created 3_h.jpg and 3_h.json
Created 4_h.jpg and 4_h.json
Created 5_h.jpg and 5_h.json
Created 6_h.jpg and 6_h.json


Created 61_h.jpg and 61_h.json
Created 62_h.jpg and 62_h.json
Created 63_h.jpg and 63_h.json
Created 64_h.jpg and 64_h.json
Created 65_h.jpg and 65_h.json
Created 66_h.jpg and 66_h.json
Created 67_h.jpg and 67_h.json
Created 68_h.jpg and 68_h.json
Created 69_h.jpg and 69_h.json


Created 7_h.jpg and 7_h.json
Created 70_h.jpg and 70_h.json
Created 71_h.jpg and 71_h.json
Created 72_h.jpg and 72_h.json
Created 73_h.jpg and 73_h.json
Created 74_h.jpg and 74_h.json
Created 75_h.jpg and 75_h.json
Created 76_h.jpg and 76_h.json
Created 77_h.jpg and 77_h.json


Created 78_h.jpg and 78_h.json
Created 79_h.jpg and 79_h.json
Created 8_h.jpg and 8_h.json
Created 80_h.jpg and 80_h.json
Created 81_h.jpg and 81_h.json
Created 82_h.jpg and 82_h.json
Created 83_h.jpg and 83_h.json
Created 84_h.jpg and 84_h.json
Created 85_h.jpg and 85_h.json


Created 86_h.jpg and 86_h.json
Created 87_h.jpg and 87_h.json
Created 88_h.jpg and 88_h.json
Created 89_h.jpg and 89_h.json
Created 9_h.jpg and 9_h.json
Created 90_h.jpg and 90_h.json
Created 91_h.jpg and 91_h.json
Created 92_h.jpg and 92_h.json
Created 93_h.jpg and 93_h.json


Created 94_h.jpg and 94_h.json
Created 95_h.jpg and 95_h.json
Created 96_h.jpg and 96_h.json
Created 97_h.jpg and 97_h.json
Created 98_h.jpg and 98_h.json
Created 99_h.jpg and 99_h.json
